## Author: Nathaniel Ruhl
### This notebook contains a proposal for the atmospheric Kalman Filter which runs parallel to the navigational Kalman Filter (NKF). These are the typical Kalman Filter equations, while the other notebook "1d_filter.ipynb" shows an idea for writing an equation for the measurement gain that we think may be appropriate based on how much the measurement disagrees with the ADM.

Upon arrival at a planetary orbit, we start by performing horizon crossings and creating the ADM. This is a callibration phase and hopefullly we can get navigational measurements from the DSN. Once the ADM is stable and the calibration phase is over, we transmition to the atmospheric filter running parralel to the NKF.

I'm not sure if the atmospheric filter is technically a "Kalman Filter" or a different kind of Filter, as the process model does not rely at all on the last state estimate, but we can still make use of the Kalman filter algorithm and exploit the optimal probability theory for state estimation when there are noisy measurements. The following statement is essentially what the atmospheric filter will do for us: "If the measured $y_{50}$ doesn't make sense with the ADM (with a known varience), then we shouldn't trust our measurement and we should rely more on the model." Specifically, the measurement could be poor due to large cross-track or radial errors.

The optimal estimate for $y_{50}$ and its varience coming out of the atmospheric filter for each iteration can be mapped into a measurement $t_{50}$ and its varience, which is the measurement inputted into the navigational filter. I am still not exactly sure how we want to deal with the varience of $t_{50}$. It might be best to map the tanh() fit into the time domain and do the curve slide/$\chi^2+1$ test, since we know this does a good job determining the precision of the measurement based on brightness. In this case, do we need to feed through the varience of the $y_{50}$ estimate into the navigational filter?

Here is an outline of the filter. The state variables of $x_k$ are the values of $y_{50}$ for different energy bands

### Process Model:
$$
\mathbf{\hat{x}_{k+1}^- = \phi s_{k+1} + y_{50}^{icp} + w_{k+1}}
$$

where $y_{50}^{icp}$ is the interccept of the ADM, the rows of $\mathbf{\phi}$ contains the coefficients of the ADM and $\mathbf{s_{k+1}}$ contains the "observation conditions/state" at time $k+1$:
$$
\mathbf{s_{k+1}} = [\alpha_{k+1}, SN_{k+1}]^T
$$

The process-noise covarience matrix is $\mathbf{Q_k}$. The error covarience of the state prediction is given by

$$
\mathbf{{P^-_{k+1}}} = E[(\mathbf{x_{k+1}}-\mathbf{\hat{x}_{k+1}^-})(\mathbf{x_{k+1}}-\mathbf{\hat{x}_{k+1}^-})^T]
$$

The process noise is directly from the varience of the ADM fit (eg. the root-mean squared error of the fit above is 1.37 km, and the standard deviation/varience will be similar):
$$
\mathbf{w_k} \sim N(0, \sigma^2_{ADM})
$$

### Measurement Model:
The measurement is described by
$$
\mathbf{z_k = y_{50,k} + v_k}
$$

where $\mathbf{v_k}$ is gaussian distributed measurement noise, with the covarience matrix $\mathbf{R_k}$:

$$
\mathbf{v_k} \sim N(0, \sigma^2_{y_{50}})
$$

$\mathbf{v_k}$ is directly related to the varience of the $y_{50}$ point, which can be determined directly by propogating errors from the tanh() fit. Based one what I've seen, this error is always about 1 km, so maybe we just give the standard deviation a value based on what we know from contructing the ADM with archived data.

### Other Kalman Filter Equations:

###### Kalman Gain:
Simplified with $\mathbf{H_k}=\mathbf{I}$ (measurement matrix for $y_{50}$):
$$
K_k = \mathbf{{P_k}^-}(\mathbf{{P_k}^-}+\mathbf{R_k})^{-1}
$$

###### State prediction:
$$
\mathbf{\hat{x}_k=\hat{x}_k^-+K_k(z_k-\hat{x}_k^-)}
$$

###### Covarience Update:
$$
\mathbf{P_k} = \left( \mathbf{I} - \mathbf{K_k} \right) \mathbf{P_k^-}
$$

There are 2 other versions of this equations if needed.

Thinking in 1d: Based on the fact that $K_k \propto P_k^-$, the measurement is weighted more when it differs from the model. I was thinking that we want the opposite behavior. I think we want $P_k \propto 1-K_k$ should only be true if $K_k \propto 1/P_k^-$